<a href="https://colab.research.google.com/github/TharunMedishetty/Diploma-Project/blob/main/Proj_29_phase_05__TharunMedishetty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the Neccessary Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from wordcloud import WordCloud
from google.colab import drive
import os

# Importing the Data

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train=pd.read_csv('/content/drive/MyDrive/Project/train.tsv',sep='\t')
test=pd.read_csv('/content/drive/MyDrive/Project/test.tsv',sep='\t')

Specifying the Train and Test sizes

In [4]:
train_size=len(train)
test_size=len(test)

In [5]:
train_data=train.copy()
test_data=test.copy()

In [6]:
train_data.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [7]:
test_data.head()

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


Creating a set combining Train & Test data. Applying Count Vectorizer on combined set will help us to get the list of all possible words.

In [8]:
combined_data = pd.concat([train_data,test_data])

In [9]:
combined_data.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,test_id
0,0.0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet,NaN
1,1.0,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,NaN
2,2.0,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,NaN
3,3.0,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,NaN
4,4.0,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity,NaN


In [10]:
# Submission set containing only the test IDs
submission = test_data[['test_id']]

In [11]:
combined_data.shape

(2175894, 9)

In [12]:
# Taking a fraction (10%) of combined data set for experimentation. Dropping train/test ids here
combined_frac = combined_data.sample(frac=0.1).reset_index(drop=True)

In [13]:
combined_frac.shape

(217589, 9)

**Advanced Text Pre-Processing**

The steps we will apply for advanced text pre-processing are:

* Removing Puncuations
* Removing Digits
* Removing Stopwords
* Changing to Lower-case words
* Lemmatization or Stemming

In [14]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [15]:
# Create a list of punctuation replacements
punctuation_symbols = []
for symbol in punctuation:
    punctuation_symbols.append((symbol, ''))

In [16]:
# Remove Punctuation
import string
def remove_punctuation(sentence: str) -> str:
    return sentence.translate(str.maketrans('', '', string.punctuation))

In [17]:
# Remove Digits
def remove_digits(x):
    x = ''.join([i for i in x if not i.isdigit()])
    return x

In [18]:
# Remove Stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
# Remove Stopwords
def remove_stop_words(x):
    x = ' '.join([i for i in x.lower().split(' ') if i not in STOPWORDS])
    return x

In [20]:
# Change to LowerCase Words
def to_lower(x):
    return x.lower()

In [21]:
# Segregating "category_name" into "category_main", "subcat_1", "subcat_2" like we did before 
def transform_category_name(category_name):
    try:
        main, sub1, sub2= category_name.split('/')
        return main, sub1, sub2
    except:
        return np.nan, np.nan, np.nan

In [22]:
train_data['category_main'], train_data['subcat_1'], train_data['subcat_2'] = zip(*train_data['category_name'].apply(transform_category_name))
cat_train = train_data[['category_main','subcat_1','subcat_2', 'price']]

In [23]:
train_data.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,category_main,subcat_1,subcat_2
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet,Men,Tops,T-shirts
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces


**Item Decription Analysis**

In [24]:
# Remove Digits, Punctuation, Stopwords, Converting to Lower-case and See the Effect
combined_data.item_description = combined_data.item_description.astype(str)
descr = combined_data[['item_description', 'price']]
descr['count'] = descr['item_description'].apply(lambda x : len(str(x)))
descr['item_description'] = descr['item_description'].apply(remove_digits)
descr['item_description'] = descr['item_description'].apply(remove_punctuation)
descr['item_description'] = descr['item_description'].apply(remove_stop_words)
descr.head(20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,item_description,price,count
0,description yet,10.0,18
1,keyboard great condition works like came box p...,52.0,188
2,adorable top hint lace key hole back pale pink...,10.0,124
3,new tags leather horses retail rm stand foot h...,35.0,173
4,complete certificate authenticity,44.0,41
5,banana republic bottoms candies skirt matching...,59.0,102
6,size small straps slightly shortened fit xs be...,64.0,83
7,get three pairs sophie cheer shorts size small...,6.0,268
8,girls size small plus green three shorts total,19.0,48
9,realized pants backwards picture dirty hand wa...,8.0,297


In [25]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
descr['item_description'] = descr['item_description'].apply(porter.stem)
descr.tail(20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,item_description,price,count
693339,beetlejuice collection collection body lotions...,NaN,87
693340,free shipping eyecat smokey eyeliner stencil tool,NaN,49
693341,macbook laptop working condition comes charg...,NaN,400
693342,please follow new listings site allows list it...,NaN,561
693343,stay day lightweight foundation little half ful,NaN,77
693344,brand new box kiehls midnight recovery concent...,NaN,154
693345,disney store exclusive marie inches long ton ...,NaN,223
693346,adidas tubular invader strap size women used box,NaN,56
693347,size im fit worn hours paid tax maci,NaN,79
693348,large strappy leggings large bling campus tee ...,NaN,99


**Handling Missing Values in Combined Data**

In [26]:
# Basic data imputation of missing values
def handle_missing_values(df):
  df['category_name'].fillna(value='missing',inplace=True)
  df['brand_name'].fillna(value='None',inplace=True)
  df['item_description'].fillna(value='None',inplace=True)

In [27]:
# Converts to Categorical Features 
def to_categorical(df):
    df['brand_name'] = df['brand_name'].astype('category')
    df['category_name'] = df['category_name'].astype('category')
    df['item_condition_id'] = df['item_condition_id'].astype('category')

In [28]:
handle_missing_values(combined_frac)
to_categorical(combined_frac)

In [29]:
handle_missing_values(combined_data)
to_categorical(combined_data)

### Remove Digits, Punctuation, Stopwords, Converting to Lower-case for combined_frac

In [30]:
combined_frac.item_description = combined_frac.item_description.astype(str)
combined_frac['item_description'] = combined_frac['item_description'].apply(remove_digits)
combined_frac['item_description'] = combined_frac['item_description'].apply(remove_punctuation)
combined_frac['item_description'] = combined_frac['item_description'].apply(remove_stop_words)
combined_frac['item_description'] = combined_frac['item_description'].apply(to_lower)
combined_frac['name'] = combined_frac['name'].apply(remove_digits)
combined_frac['name'] = combined_frac['name'].apply(remove_punctuation)
combined_frac['name'] = combined_frac['name'].apply(remove_stop_words)
combined_frac['name'] = combined_frac['name'].apply(to_lower)
combined_frac.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,test_id
0,NaN,ing black bodysuit,1,"Women/Tops & Blouses/Tank, Cami",None,NaN,1,ing black bodysuit nwt x hand wash rayon spa...,488168.0
1,NaN,bundle new mascaras,1,Beauty/Makeup/Eyes,Urban Decay,NaN,0,travel friendly size brand new never opened sh...,358535.0
2,4192.0,rose gold apple watch mm,3,Women/Women's Accessories/Watches,None,231.0,0,excellent condition comes larger band box char...,NaN
3,NaN,lululemon athletica bra tank top size,3,Women/Athletic Apparel/Shirts & Tops,lululemon athletica,NaN,0,lululemon athletica bra tank top size great c...,393774.0
4,831883.0,open leg maxi dress,3,Women/Dresses/Full-Length,FOREVER 21,9.0,1,slit maxi dress slit left side navydark blue s...,NaN


### Remove Digits, Punctuation, Stopwords, Converting to Lower-case for combined_data

In [31]:
combined_data.item_description = combined_data.item_description.astype(str)
combined_data['item_description'] = combined_data['item_description'].apply(remove_digits)
combined_data['item_description'] = combined_data['item_description'].apply(remove_punctuation)
combined_data['item_description'] = combined_data['item_description'].apply(remove_stop_words)
combined_data['item_description'] = combined_data['item_description'].apply(to_lower)
combined_data['name'] = combined_data['name'].apply(remove_digits)
combined_data['name'] = combined_data['name'].apply(remove_punctuation)
combined_data['name'] = combined_data['name'].apply(remove_stop_words)
combined_data['name'] = combined_data['name'].apply(to_lower)
combined_data.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,test_id
0,0.0,mlb cincinnati reds shirt size xl,3,Men/Tops/T-shirts,None,10.0,1,description yet,NaN
1,1.0,razer blackwidow chroma keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,keyboard great condition works like came box p...,NaN
2,2.0,avaviv blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,adorable top hint lace key hole back pale pink...,NaN
3,3.0,leather horse statues,1,Home/Home Décor/Home Décor Accents,None,35.0,1,new tags leather horses retail rm stand foot h...,NaN
4,4.0,k gold plated rose,1,Women/Jewelry/Necklaces,None,44.0,0,complete certificate authenticity,NaN


Applying CountVectorizer / TfidfVectorizer / LabelBinarizer
* CountVectorizer counts word frequencies.
* TF-IDF Vectorizer gives more significance (puts more weights) on rare words, and less significance (puts lesser weights) on frequent words.
* Label Binarizer converts labels into numeric representations for e.g. "A,B,C" -> [1,2,3]

In [32]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer

In [33]:
# Apply Count Vectorizer to "name", this converts it into a sparse matrix 
cv = CountVectorizer(min_df=10)
X_name = cv.fit_transform(combined_data['name'])

In [34]:
# Apply Count Vectorizer to "category_name", this converts it into a sparse matrix
cv = CountVectorizer()
X_category = cv.fit_transform(combined_data['category_name'])

In [35]:
# Apply TFIDF to "item_description", 
tv = TfidfVectorizer(max_features=55000, ngram_range=(1, 2), stop_words='english')
X_description = tv.fit_transform(combined_data['item_description'])

In [36]:
# Apply LabelBinarizer to "brand_name"
lb = LabelBinarizer(sparse_output=True)
X_brand = lb.fit_transform(combined_data['brand_name'])

In [37]:
# vstack - adds rows
# hstack - adds columns
# csr_matrix - handles sparse matrix

from scipy.sparse import vstack, hstack, csr_matrix
X_dummies = csr_matrix(pd.get_dummies(combined_data[['item_condition_id', 'shipping']], sparse=True).values)

In [38]:
# Create the final sparse matrix combining everything together
sparse_merge = hstack((X_dummies, X_description, X_brand, X_category, X_name)).tocsr()

**Splitting Train and Test Data**

In [39]:
X_train_sparse = sparse_merge[:train_size]
X_test = sparse_merge[train_size:]

**Creating validation data using train data**

In [40]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3, shuffle=True, random_state=12345)
y = np.log1p(train_data['price'])
i = 0;
for train_indicies, valid_indicies in kf.split(X_train_sparse):
    X_train, y_train = X_train_sparse[train_indicies], y[train_indicies]
    X_valid, y_valid = X_train_sparse[valid_indicies], y[valid_indicies]

**Funtion to Run the Model**

In [41]:
from sklearn.metrics import mean_squared_error, r2_score
def run_model_advText(model, X_train, y_train, X_valid, y_valid, verbose = False):
    model.fit(X_train, y_train)
    preds_valid = model.predict(X_valid)
    mse = mean_squared_error(y_valid,preds_valid)
    r_sq = r2_score(y_valid,preds_valid)
    print("Mean Squared Error Value : "+"{:.2f}".format(mse))
    print("R-Squared Value : "+"{:.2f}".format(r_sq))
    return model, mse, r_sq

In [42]:
from sklearn import linear_model
import lightgbm
import xgboost

**Storing in pickle file**

In [43]:
regression_model=linear_model.Ridge(solver = "saga", fit_intercept=False)
regression_model.fit(X_train, y_train)

Ridge(fit_intercept=False, solver='saga')

In [44]:
import pickle 
pickle_out = open("Regression_Model.pkl", mode = "wb") 
pickle.dump(regression_model, pickle_out) 
pickle_out.close()

**Model-1: Ridge Regression**

In [ ]:
ridge_reg = linear_model.Ridge(solver = "saga", fit_intercept=False)
print("Ridge Regression (After advanced Text Pre-processing)")
print("-----------------------------------------------------")
model_11, mse_11, r_sq_11 = run_model_advText(ridge_reg, X_train, y_train, X_valid, y_valid)

Ridge Regression (After advanced Text Pre-processing)
-----------------------------------------------------
Mean Squared Error Value : 0.22
R-Squared Value : 0.60


In [ ]:
import math

In [ ]:
print("RMSE:",math.sqrt(mse_11))

RMSE: 0.4740167248352686


**Model-2: LGBM Regression**

In [ ]:
lgbm_reg = lightgbm.LGBMRegressor()
print("LGBM Regression (After advanced Text Pre-processing)")
print("----------------------------------------------------")
model_22, mse_22, r_sq_22 = run_model_advText(lgbm_reg, X_train, y_train, X_valid, y_valid)

LGBM Regression (After advanced Text Pre-processing)
----------------------------------------------------
Mean Squared Error Value : 0.29
R-Squared Value : 0.48


In [ ]:
print("RMSE:",math.sqrt(mse_22))

RMSE: 0.5398682932439914


**Model-3: XGB Regression** 

In [ ]:
xgb_params = {'n_estimators':500, 'max_depth':8}
xgb_reg = xgboost.XGBRegressor(**xgb_params)
print("XGB Regression (After advanced Text Pre-processing)")
print("---------------------------------------------------")
model_33, mse_33, r_sq_33 = run_model_advText(xgb_reg, X_train, y_train, X_valid, y_valid)

XGB Regression (After advanced Text Pre-processing)
---------------------------------------------------
[18:49:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Mean Squared Error Value : 0.25
R-Squared Value : 0.55


In [ ]:
print("RMSE:",math.sqrt(mse_33))

RMSE: 0.5030881375121298


**Results without Advanced Text Pre-Processing**

In [ ]:
data={'Model': ['Ridge', 'LGBM', 'XGB'],
        'MSE': [1464.71, 1035.26, 893.99],
      'R-Suared':[0.02,0.31,0.40]}
results_without_Adv_txt=pd.DataFrame(data)

In [ ]:
results_without_Adv_txt

,Model,MSE,R-Suared
0,Ridge,1464.71,0.02
1,LGBM,1035.26,0.31
2,XGB,893.99,0.40


In [ ]:
data={'Model': ['Ridge', 'LGBM', 'XGB'],
        'MSE': [0.22, 0.29, 0.25],
      'R-Suared':[0.60,0.48,0.55]}
results_with_Adv_txt=pd.DataFrame(data)

In [ ]:
results_with_Adv_txt

,Model,MSE,R-Suared
0,Ridge,0.22,0.60
1,LGBM,0.29,0.48
2,XGB,0.25,0.55


Improvement in Model Performance after using Advanced Text Pre-Processing :
* MSE has now decreased for all models
* 𝑅2  value has now increased for all models
However, for Ridge model, the improvement seems the highest. Hence, we will choose Ridge to apply on the test dataset.

In [ ]:
mse_before = [1464.71, 1035.26, 893.99]
r_sq_before = [0.02,0.31,0.40]
mse_after = [mse_11, mse_22, mse_33]
r_sq_after = [r_sq_11, r_sq_22, r_sq_33]
model_data = {'Model': ['Ridge','LGBM','XGB'],
              'MSE_without_Advanced_TextProcessing': mse_before,
              'R_Square_without_Advanced_TextProcessing': r_sq_before,
              'MSE_with_Advanced_TextProcessing': mse_after,
              'R_Square_with_Advanced_TextProcessing': r_sq_after}
data_compare = pd.DataFrame(model_data)

In [ ]:
data_compare

,Model,MSE_without_Advanced_TextProcessing,R_Square_without_Advanced_TextProcessing,MSE_with_Advanced_TextProcessing,R_Square_with_Advanced_TextProcessing
0,Ridge,1464.71,0.02,0.224692,0.599285
1,LGBM,1035.26,0.31,0.291458,0.480214
2,XGB,893.99,0.40,0.253098,0.548626


**Apply Final Ridge Model on Test Dataset and Submission :**

In [ ]:
predictions = ridge_reg.predict(X_test)
submission["price"] = np.expm1(predictions)
submission.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,test_id,price
0,0,10.768500
1,1,11.873002
2,2,47.088323
3,3,16.253206
4,4,6.561265


In [ ]:
submission.to_csv("submission.csv", index = False)

In [ ]:
submission

,test_id,price
0,0,10.768500
1,1,11.873002
2,2,47.088323
3,3,16.253206
4,4,6.561265
...,...,...
693354,693354,21.179670
693355,693355,31.292689
693356,693356,6.536530
693357,693357,15.416135


**Merging Price Column to the Test Data**

In [ ]:
test_data_price=test_data.merge(submission, left_on='test_id', right_on='test_id',
          suffixes=('_left', '_right'))

In [ ]:
test_data_price

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description,price
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7,10.768500
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined...",11.873002
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...,47.088323
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...,16.253206
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...,6.561265
...,...,...,...,...,...,...,...,...
693354,693354,Quartz crystal on Flint stone,1,Home/Home Décor/Home Décor Accents,NaN,0,Flint/Quartz cluster. Self mined ✨measures 3x2...,21.179670
693355,693355,It Cosmetics - Travel Bundle,1,Beauty/Makeup/Makeup Sets,IT Cosmetics,1,It Cosmetics travel bundle. Includes: Brow pow...,31.292689
693356,693356,Galaxy S8 hard shell case,1,"Electronics/Cell Phones & Accessories/Cases, C...",NaN,1,New. Free shipping Basstop case,6.536530
693357,693357,Hi low floral kimono,2,Women/Swimwear/Cover-Ups,NaN,0,Floral kimono. Tropical print. Open front. Hi ...,15.416135
